## 1. Library Import

In [94]:
import numpy as np
import time
import itertools

__author__ = "Marco Odehnal"
__copyright__ = "Copyright 2018"
__status__ = "Prototype"

## 2. Problem formulation

In [99]:
# The cities for the problem, taken from att48
cities = np.matrix([[1, 6734, 1453],
                    [2, 2233, 10],
                    [3, 5530, 1424],
                    [4, 401, 841],
                    [5, 3082, 1644],
                    [6, 7608, 4458],
                    [7, 7573, 3716],
                   [8, 123, 8526],
                   [9, 4356, 376],
                   [10, 7456, 236],
                    [11, 67534, 14553],
                    [2, 22353, 150],
                    [3, 55530, 14524],
                    [4, 45501, 841],
                    [5, 350852, 16544],
                    [6, 76508, 4458],
                    [7, 75573, 35716],
                   [8, 1253, 85526],
                   [9, 43556, 5376],
                   [10, 57456, 2536]                   ])

cities = np.matrix([[1, 6734, 1453],
                    [2, 2233, 10],
                    [3, 5530, 1424],
                    [4, 401, 841],
                    [5, 3082, 1644],
                    [6, 7608, 4458],
#                     [7, 7573, 3716],
#                    [8, 123, 8526],
#                    [9, 4356, 376],
#                    [10, 7456, 236],
#                     [11, 67534, 14553],
#                     [2, 22353, 150],
#                     [3, 55530, 14524],
#                     [4, 45501, 841],
#                     [5, 350852, 16544],
#                     [6, 76508, 4458],
                    [7, 75573, 35716],
                   [8, 1253, 85526],
                   [9, 43556, 5376],
                   [10, 57456, 2536]                   ])


# cities = np.matrix([
#   [1,6734,1453],
#   [2,2233,10],
#   [3,5530,1424],
#   [4,401,841],
#   [5,3082,1644],
#   [6,7608,4458],
#   [7,7573,3716],
#   [8,7265,1268],
#   [9,6898,1885],
#   [10,1112,2049],
#   [11,5468,2606],
#   [12,5989,2873],
#   [13,4706,2674],
#   [14,4612,2035],
#   [15,6347,2683],
#   [16,6107,669],
#   [17,7611,5184],
#   [18,7462,3590],
#   [19,7732,4723],
#   [20,5900,3561],
#   [21,4483,3369],
#   [22,6101,1110],
#   [23,5199,2182],
#   [24,1633,2809],
#   [25,4307,2322],
#   [26,675,1006],
#   [27,7555,4819],
#   [28,7541,3981],
#   [29,3177,756],
#   [30,7352,4506],
#   [31,7545,2801],
#   [32,3245,3305],
#   [33,6426,3173],
#   [34,4608,1198],
#   [35,23,2216],
#   [36,7248,3779],
#   [37,7762,4595],
#   [38,7392,2244],
#   [39,3484,2829],
#   [40,6271,2135],
#   [41,4985,140],
#   [42,1916,1569],
#   [43,7280,4899],
#   [44,7509,3239],
#   [45,10,2676],
#   [46,6807,2993],
#   [47,5185,3258],
#   [48,3023,1942]
# ])

# The number of verteces 
n = np.size(cities,0)

# Adjacency matrix
# NOT OPTIMAL GENERATION, JUST FOR TESTING
M = np.matrix([[np.linalg.norm(cities[i,1:3]-cities[j,1:3]) for j in range(n)] for i in range(n)])

M[M==0] = np.inf

# M = M[:10,:10]
# print(M)

## 3. Classic approach

In [87]:
# To run the classical Branch and Bound method, use this function.
def BranchAndBound(graph, timed = False):

#     adj_mat = graph.weighted_adjacency_matrix
    adj_mat = graph
    adj_mat = np.matrix(adj_mat)
    
    n = np.size(adj_mat,1)

    for i in range(0,n):
        adj_mat[i,i] = np.inf
    
    rem_nodes = list(range(2,n+1))
    cost = 0
    optimal_cost = np.inf
    v = [1] 
    
    t1 = time.time()
    cost, path, opt_cost, opt_path, duration = BnBClassic(adj_mat,v,rem_nodes,cost,optimal_cost, [], timed, t1)
        
    return opt_cost, opt_path, duration
    
        
def BnBClassic(A,v,rem_nodes, cost, optimal_cost, opt_path, timed, t1):
    
    # YOU CAN CHANGE THE MAXIMUM TIME HERE
    if timed:
        if time.time()-t1 > 6:
            return cost, v, optimal_cost, opt_path, time.time() - t1
    
    # End of the path
    if len(rem_nodes)==1:
        cost += A[v[-1]-1, rem_nodes[0]-1] + A[rem_nodes[0]-1,0]
        v += rem_nodes + [1]

        if cost < optimal_cost:
            return cost, v, cost, v, time.time() - t1
        else:
            return cost, v, optimal_cost, opt_path, time.time() - t1
    
    else:
        
        k = len(v)
        n = np.size(A,0)+1

        cost_branches = []
        path_branches = []
        f = []
        
        # Calculating the costs, the f and the paths
        for i in rem_nodes:
            cost_branches.append(cost + A[v[-1]-1,i-1])
            path_branches.append(v+[i])
            # In g we select the minimum distance we can be able to travel
            g = np.min(A[:,[x-1 for x in rem_nodes + [1] if x!= i]])
            if g == np.inf:
                continue
            f.append(cost_branches[-1] + (n-k)*
                     g)
        
        if len(f) == 0:
            return cost, v, optimal_cost, opt_path, time.time() - t1
        
        # Sorting the arrays
        order = np.argsort(f)
        cost_branches = [cost_branches[i] for i in order]
        path_branches = [path_branches[i] for i in order]
        f = [f[i] for i in order] 


        # We explore recursively the branches and check if an optimal solution can be found
        for i in range(len(f)):
            
            # We discard all of the branches that cannot decrease the cost function
            # As all of the branches are sorted by cost, the following branches after
            # a discarded one will also be discarded
            if f[i] >= optimal_cost:
                break
            else:
                rem_nodes_sub = [x for x in rem_nodes if x not in path_branches[i]]
                cost, v, optimal_cost, opt_path, duration = BnBClassic(A,path_branches[i],rem_nodes_sub, cost_branches[i], optimal_cost, opt_path, timed, t1)
            
        return cost, v, optimal_cost, opt_path, time.time()-t1
    
print(BranchAndBound(M, timed = True))

(30267.626621916959, [1, 10, 3, 9, 5, 2, 4, 8, 6, 7, 1], 4.0778892040252686)


In [100]:
# M = np.matrix([[np.inf, 3, 93, 13, 33, 9, 57,34],
#                [4, np.inf, 77, 42, 21, 16, 34,72],
#                [45, 17, np.inf, 36, 16, 28, 25,31],
#                [39, 90, 80, np.inf, 56, 7, 91, 94],
#                [28, 46, 88, 33, np.inf, 25, 57,23],
#                [3, 88, 18, 46, 92, np.inf, 7,11],
#                [44, 26, 33, 27, 84, 39, np.inf,15],
#                [55, 12, 57, 79, 32, 44, 41, np.inf]])

# M = np.matrix([[np.inf,10,25,40,60,20],
#          [10,np.inf,30,80,70,100],
#          [25,30,np.inf,45,85,90],
#          [40,80,45,np.inf,60,80],
#          [60,70,85,60,np.inf,70],
#          [20,100,90,80,70,np.inf]])

# M = np.matrix([[np.inf,2,3,5],
#              [2,np.inf,6,1],
#              [3,6,np.inf,4],
#              [5,1,4,np.inf]])




n = np.size(M[0:15,0:15],1)

rem_nodes = list(range(2,n+1))
cost = 0
optimal_cost = np.inf
v = [1]

# Note: the third variable optimal cost is used to determine if we have reached the end of a branch

def BranchAndBound(graph, timed = False):

    np.size(graph,1)

    rem_nodes = list(range(2,n+1))
    cost = 0
    optimal_cost = np.inf
    v = [1] 
    
    if timed:
        cost, path, opt_cost, opt_path = BnBClassic(M[0:15][0:15],v,rem_nodes,cost,optimal_cost, [], timed, t1, t2)
    else:
        cost, path, opt_cost, opt_path = BnBClassic(M[0:15][0:15],v,rem_nodes,cost,optimal_cost, [])
    
        
def BnBClassic(A,v,rem_nodes, cost, optimal_cost, opt_path, timed, t1, t2 = time.time()):
    
    # Time measure
    if timed:
        if t2-t1 > 10:
            return cost, v, optimal_cost, opt_path
    
    # End of the path
    if len(rem_nodes)==1:
        cost += A[v[-1]-1, rem_nodes[0]-1] + A[rem_nodes[0]-1,0]
        v += rem_nodes + [1]

        if cost < optimal_cost:
            return cost, v, cost, v
        else:
            return cost, v, optimal_cost, opt_path
    
    else:
        
        k = len(v)
        n = np.size(A,0)+1

        cost_branches = []
        path_branches = []
        f = []
        
        # Calculating the costs, the f and the paths
        for i in rem_nodes:
            cost_branches.append(cost + A[v[-1]-1,i-1])
            path_branches.append(v+[i])
            # In g we select the minimum distance we can be able to travel
            g = np.min(A[:,[x-1 for x in rem_nodes + [1] if x!= i]])
            if g == np.inf:
                continue
            f.append(cost_branches[-1] + (n-k)*
                     g)
        
        if len(f) == 0:
            return cost, v, optimal_cost, opt_path
        
        # Sorting the arrays
        order = np.argsort(f)
        cost_branches = [cost_branches[i] for i in order]
        path_branches = [path_branches[i] for i in order]
        f = [f[i] for i in order] 


        # We explore recursively the branches and check if an optimal solution can be found
        for i in range(len(f)):
            
            # We discard all of the branches that cannot decrease the cost function
            # As all of the branches are sorted by cost, the following branches after
            # a discarded one will also be discarded
            if f[i] >= optimal_cost:
                break
            else:
                rem_nodes_sub = [x for x in rem_nodes if x not in path_branches[i]]
                cost, v, optimal_cost, opt_path = BnBClassic(A,path_branches[i],rem_nodes_sub, cost_branches[i], optimal_cost, opt_path)
            
        return cost, v, optimal_cost, opt_path

t1 = time.time()     

cost, path, opt_cost, opt_path = BnBClassic(M[0:15][0:15],v,rem_nodes,cost,optimal_cost, [])

t2= time.time()

print('Time:', t2-t1)    
print('Minimum_cost:',opt_cost)
print('Best path:',opt_path)    

TypeError: BnBClassic() missing 2 required positional arguments: 'timed' and 't1'

## 4. Adding and removing edges (binary tree)

The following cell contains helper functions used in the main algorithm, the main function is defined after the following cell.

In [101]:
# Helper functions used in the following algorithm

# This function calculates the normaliced matrix with at least one zero in each row and column.
# It also computes the cost of reduction.
def mat_redux(M, cost):
    
    # Used to reshape the minimum value arrays into matrices
    n = np.size(M,1)    
    
    # Reduction by rows
    m1 = M.min(axis=1)
    # to avoid inf-inf math error
    m1[m1 == np.inf] = 0
    Mrow = M - m1.reshape(n,1)
    
    # Reduction by columns
    m2 = Mrow.min(axis=0)
    m2[m2 == np.inf] = 0
    Mcol = Mrow - np.matrix(m2).reshape(1,n)
    
    # Lower bound calculation
    f = m1.sum() + m2.sum()
    
    return cost + f, Mcol

# This function selects the zero corresponding to the edge with the greatest weight
def select_edges(A, n):

    # We wish to know where are the pivots
    zeros_irow, zeros_icol = np.where(A==0)

    # Variables initialization
    max_edge_cost = 0
    opt_edge = (0,0)

    #  We traverse every zero such that it has the highest cost
    for k in range(len(zeros_irow)):
        
        i = zeros_irow[k]
        j = zeros_icol[k]
        x = [x for x in range(0,n) if x != i]
        y = [x for x in range(0,n) if x != j]

        edge_cost = np.min(A[i,y]) + np.min(A[x,j])   
        
        if edge_cost > max_edge_cost:
            
            max_edge_cost = edge_cost
            opt_edge = (i,j)

    return opt_edge

# This function created the children Left and Right subtree (with and without the edge)
def children_subtrees(M, edge, f):
    
    # Left and Right matrices
    L = np.copy(M)
    R = np.copy(M)

    # For the left child we add infinity to the column and row corresponding to the chosen edge
    try:
        L[edge[0],:] = np.inf
        L[:,edge[1]] = np.inf
        L[edge[1],edge[0]] = np.inf
    except:
        print(edge)
    # After this the matrix should be reduced
    f_L, L = mat_redux(L,f)

    # For the right child we just add infinity to the location of the node, so that it's excluded
    R[edge[0], edge[1]] = np.inf
    f_R, R = mat_redux(R,f)

    return f_L, L, f_R, R

# Function used to reconstruct the path from the given edges 
# (the edge list is assumed to be same size of the number of cities)
def create_path_from_edges(EdgeList):
    
    E = EdgeList + []
    
    n = len(E)
    path = [1]
    possible = True

    while True:

        continue_chain = 0

        for i in E:
            if i[0] == path[-1]-1:
                path.append(i[1]+1)
                continue_chain = 1
                break

        E.remove(i)

        if continue_chain == 0:
            return []
        if len(path) == n:
            return path

    

In [102]:
# M = np.matrix([[np.inf, 3, 93, 13, 33, 9, 57],
#                [4, np.inf, 77, 42, 21, 16, 34],
#                [45, 17, np.inf, 36, 16, 28, 25],
#                [39, 90, 80, np.inf, 56, 7, 91],
#                [28, 46, 88, 33, np.inf, 25, 57],
#                [3, 88, 18, 46, 92, np.inf, 7],
#                [44, 26, 33, 27, 84, 39, np.inf]])

# M = np.matrix([[np.inf,2,3,5],
#              [2,np.inf,6,1],
#              [3,6,np.inf,4],
#              [5,1,4,np.inf]])

# The algorithm
def BnBBinaryTree(M):
    
    # Matrix reduction step
    f,MR = mat_redux(M,0)
    
    n = np.size(M,1)

    cost, edges_col, opt_edge_col = BnBBinaryTreeRecursive(MR,[],f,np.inf, [], n)
    
    return cost, create_path_from_edges(opt_edge_col)+[1]

# This is the recursive algorithm that is called after pre-processing the matrix    
def BnBBinaryTreeRecursive(M, edges_collection, cost, opt_cost, opt_edge_col, n):
    
    # If the optimum cost is less than the actual cost we are not supposed to do anything
    if opt_cost <= cost:
        return opt_cost, edges_collection, opt_edge_col
    
    # We know that we cannot find a solution if there are less nodes than cities to visit
    if n-len(edges_collection) > np.sum(M.min(axis=1) == 0):
        return opt_cost, edges_collection, opt_edge_col
    
    # If the search is complete, a solution has been found
    elif n-len(edges_collection) == np.sum(M.min(axis=1) == 0) == 0:
        if cost < opt_cost:
            if len(create_path_from_edges(edges_collection)) != 0:
                return cost, edges_collection, edges_collection
            else:
                return opt_cost, edges_collection, opt_edge_col
        else:
            return opt_cost, edges_collection, opt_edge_col
    
    # Edge selection
    edge = select_edges(M, n, )
    
    if edge == (0,0):
        return opt_cost, edges_collection, opt_edge_col
    
    # Children trees creation
    f_L, L, f_R, R = children_subtrees(M, edge, cost, )
    
    # We search the trees
    opt_cost, edges_col, opt_edge_col = BnBBinaryTreeRecursive(L, edges_collection + [edge], f_L, opt_cost, opt_edge_col, n)
    if opt_cost > f_R:
        opt_cost, edges_col, opt_edge_col = BnBBinaryTreeRecursive(R, edges_collection, f_R, opt_cost, opt_edge_col, n)
    return opt_cost, edges_col, opt_edge_col


t1 = time.time()
cost, path = BnBBinaryTree(M)

t2= time.time()
print('Time:', t2-t1)    
print('Minimum_cost:',cost)
print('Best path:',path)

Time: 1.0317189693450928
Minimum_cost: 272752.773152
Best path: [1, 6, 9, 10, 7, 8, 4, 2, 5, 3, 1]


In [67]:
E = [(1, 3), (2, 0), (0, 1), (3, 2)]

n = len(E)

path = [1]

possible = True

while True:
    
    continue_chain = 0
    
    for i in E:
        if i[0] == path[-1]-1:
            path.append(i[1]+1)
            continue_chain = 1
            break
            
    E.remove(i)
            
    if continue_chain == 0:
        print('noooo')
    if len(path) == n:
        print(path)
        break
            

the array [(1, 3), (2, 0), (0, 1), (3, 2)]
current path [1, 2]
last element (0, 1)
the array [(1, 3), (2, 0), (3, 2)]
current path [1, 2, 4]
last element (1, 3)
the array [(2, 0), (3, 2)]
current path [1, 2, 4, 3]
last element (3, 2)
[1, 2, 4, 3]


In [97]:
print(BranchAndBoundBinaryTree(M, timed = True))

(inf, [1], 360.00139021873474)
